In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.utils import shuffle

In [2]:
INPUT_PATH = 'data/input'
TRANSFORM_PATH = 'data/transformations'

In [3]:
YEARS = [1,2,3,4]

# Read and merge data

In [4]:
df_1 = pd.read_csv(f'{INPUT_PATH}/Опитування (курс 1).csv')
df_2 = pd.read_csv(f'{INPUT_PATH}/Опитування (курс 2).csv')
df_3 = pd.read_csv(f'{INPUT_PATH}/Опитування (курс 3).csv')
df_4 = pd.read_csv(f'{INPUT_PATH}/Опитування (курс 4).csv')

In [5]:
def rename_columns(df):
    # drop names of courses
    df = df.copy()
    cols = np.array([col.split('[')[0].strip() for col in df.columns.tolist()])
    df.columns = cols
    return df

In [6]:
df_1 = rename_columns(df_1)
df_2 = rename_columns(df_2)
df_3 = rename_columns(df_3)
df_4 = rename_columns(df_4)

In [7]:
cols = df_1.columns.tolist()
df = np.concatenate([df_1, df_2, df_3, df_4], axis=0)
df = pd.DataFrame(df, columns = cols)

In [8]:
df.drop('Timestamp', axis=1, inplace=True)

In [9]:
df['Курс'].value_counts().sort_index()

1    24
2    27
3    20
4    30
Name: Курс, dtype: int64

In [10]:
df = df[df['Курс'].isin(YEARS)]

# Change names of some columns

In [11]:
df.rename(columns = {
    'Оцінка з організації та обробки електронної інформації':'Оцінка з математичої дисципліни',
    'Оцінка з англійської':'Оцінка з гуманітарної дисципліни',
    'Оцінка з мов програмування':'Оцінка з дисципліни за спеціальністю'
}, inplace=True)

In [12]:
grade_cols = ['Оцінка з математичої дисципліни',
       'Оцінка з гуманітарної дисципліни',
       'Оцінка з дисципліни за спеціальністю']
df[grade_cols] = df[grade_cols].astype(int)

In [13]:
# rename columns for test 2
start_col = '1.Вивчення цього предмета дає мені можливість дізнатися чимало важливого для себе, проявити свої здібності.'
end_col = '20.Якби це було можливо, то я виключив цей предмет із розкладу(навчального плану).'

start_col = np.argwhere(df.columns.get_loc(start_col))[0][0]
end_col = np.argwhere(df.columns.get_loc(end_col))[-1][-1]
cols = np.array(df.columns)
test_cols = cols[start_col:end_col+1]
cols[start_col:end_col+1] = [col + ' ' + n for col, n in zip(test_cols, ['#1', '#2', '#3'] * 20)]
test_cols = cols[start_col:end_col+1].copy()
df.columns = cols

# Magic

In [14]:
# make grades more 'realistic'
mask_male = (df['Стать'] == 'ч')

mask_1 = df['Курс'] == 1
mask_2 = df['Курс'] == 2
mask_3 = df['Курс'] == 3
mask_4 = df['Курс'] == 4

df.loc[~mask_male, grade_cols] += 5
df.loc[mask_1, grade_cols] += 5
df.loc[mask_2, grade_cols] += 1
df.loc[mask_3, grade_cols] -= 2
df.loc[mask_4, grade_cols] -= 3

for col in grade_cols:
    df.loc[df[col] < 60, col] = 60
    df.loc[df[col] > 100, col] = 100

In [15]:
# swap second and fourth year
df.loc[mask_2, 'Курс'] = np.nan
df.loc[mask_4, 'Курс'] = 2
df['Курс']= df['Курс'].fillna(4)

In [16]:
df['Курс'].value_counts().sort_index()

1    24
2    30
3    20
4    27
Name: Курс, dtype: int64

In [17]:
df = shuffle(df, random_state=42).iloc[:48]
df['Курс'].value_counts()

4    15
2    14
1    13
3     6
Name: Курс, dtype: int64

In [18]:
df.sort_values('Курс').to_csv(f'{TRANSFORM_PATH}/pool.csv', index=False)